# Large Survey DataBase

LSDB (Large Survey DataBase) is a framework designed for fast spatial analysis of extremely large astronomical catalogs, enabling efficient querying and crossmatching of over a billion sources. It specifically addresses large-scale data processing challenges, particularly those posed by LSST. Read more about LSDB here.
LSDB is a framework to facilitate and enable spatial analysis for extremely large astronomical databases (i.e. querying and crossmatching O(1B) sources). This package uses dask to parallelize operations across multiple HiPSCat partitioned surveys. 


Please check out:

 - [github.com/astronomy-commons/lsdb](github.com/astronomy-commons/lsdb)
 - [lsdb.io](lsdb.io) (Check out this! There are multiple links to already partitioned catalogs, like GaiaDR3, ZTF and others).

We are applying on S-PLUS a new server side application with lsdb. It allows adding constrains on queries over the network, like selecting specific columns, and filtering results. All the examples below follow this implementation.

We have a walkthrough on how to use LSDB with S-PLUS data available at [splus.cloud/documentation/lsdb_query](https://splus.cloud/documentation/lsdb_query).


### Lets import splusdata and lsdb

In [2]:
import lsdb
import splusdata

Using splusdata we can print the available catalogs, and get the link for one in particular.

In [6]:
# This will print all available links
print(splusdata.get_hipscats())

# This will get the link for a specific pattern
print(splusdata.get_hipscats("dr4/dual"))


{'dr4_vacs': {'qso_z': {'hipscats': ['qso_z'], 'margins': ['qso_z_2arcsec']}, 'photozs': {'hipscats': ['photozs'], 'margins': ['photozs_2arcsec']}, 'calib_flag': {'hipscats': ['calib_flag'], 'margins': ['calib_flag_2arcsec']}, 'sqg': {'hipscats': ['sqg'], 'margins': ['sqg_2arcsec']}}, 'sdr1': {'hipscats': ['shorts'], 'margins': ['shorts_2arcsec']}, 'dr4': {'hipscats': ['single', 'dual', 'psf'], 'margins': ['psf_2arcsec', 'single_2arcsec', 'dual_2arcsec']}}
[['https://splus.cloud/HIPS/catalogs/dr4/dual/', 'https://splus.cloud/HIPS/catalogs/dr4/dual_2arcsec/']]


### Important

Before getting into the code, start a dask Client with the desired specs, this is what does the "parallel magic". This step is very important. It will also print a url to a dashboard that you can use to check the status of your queries.

In [4]:
from dask.distributed import Client

client = Client(n_workers=10, memory_limit="8GB")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 20,Total memory: 74.51 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60380,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 74.51 GiB
Comm: tcp://127.0.0.1:60407,Total threads: 2
Dashboard: http://127.0.0.1:60410/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:60383,


2024-09-17 01:34:41,811 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 12 memory: 158 MB fds: 279>>
Traceback (most recent call last):
  File "/Users/gustavoschwarz/miniconda3/lib/python3.12/site-packages/tornado/ioloop.py", line 937, in _run
    val = self.callback()
          ^^^^^^^^^^^^^^^
  File "/Users/gustavoschwarz/miniconda3/lib/python3.12/site-packages/distributed/system_monitor.py", line 168, in update
    net_ioc = psutil.net_io_counters()
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gustavoschwarz/miniconda3/lib/python3.12/site-packages/psutil/__init__.py", line 2139, in net_io_counters
    rawdict = _psplatform.net_io_counters()
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 12] Cannot allocate memory
2024-09-17 09:01:53,395 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 6 memory: 403 MB fds: 281>>
Traceback (most recent

### Exploring the data

Before we start querying the data, let's take a look at the available columns in the catalog. 
This is important to know what we can query and what we can get from the database. Filtering the columns will make the query faster and more efficient.

In [ ]:
# This will return a list because there may be more than
# one match for dr4/dual
dr4_dual_link = splusdata.get_hipscats("dr4/dual")[0]

# dr4_dual_link will be a tuple, because of the margin link.
dual_link = lsdb.read_hipscat(dr4_dual_link[0])

In [5]:
list(dual_link.columns)

['ID', 'RA', 'DEC', 'A', 'B', 'BACKGROUND', 'BACKGROUND_J0378', 'BACKGROUND_J0395', 'BACKGROUND_J0410', 'BACKGROUND_J0430', 'BACKGROUND_J0515', 'BACKGROUND_J0660', 'BACKGROUND_J0861', 'BACKGROUND_g', 'BACKGROUND_i', 'BACKGROUND_r', 'BACKGROUND_u', 'BACKGROUND_z', 'CLASS_STAR', 'DET_ID_dual', 'EBV_SCH', 'ELLIPTICITY', 'ELONGATION', 'FLUX_RADIUS_20', 'FLUX_RADIUS_50', 'FLUX_RADIUS_70', 'FLUX_RADIUS_90', 'FWHM', 'FWHM_J0378', 'FWHM_J0395', 'FWHM_J0410', 'FWHM_J0430', 'FWHM_J0515', 'FWHM_J0660', 'FWHM_J0861', 'FWHM_g', 'FWHM_i', 'FWHM_n', 'FWHM_n_J0378', 'FWHM_n_J0395', 'FWHM_n_J0410', 'FWHM_n_J0430', 'FWHM_n_J0515', 'FWHM_n_J0660', 'FWHM_n_J0861', 'FWHM_n_g', 'FWHM_n_i', 'FWHM_n_r', 'FWHM_n_u', 'FWHM_n_z', 'FWHM_r', 'FWHM_u', 'FWHM_z', 'Field', 'ID_DEC', 'ID_RA', 'ISOarea', 'J0378_ID_dual', 'J0378_PStotal', 'J0378_aper_3', 'J0378_aper_6', 'J0378_auto', 'J0378_iso', 'J0378_petro', 'J0395_ID_dual', 'J0395_PStotal', 'J0395_aper_3', 'J0395_aper_6', 'J0395_auto', 'J0395_iso', 'J0395_petro', 'J

### Querying the data

Here we will show some examples of queries that can be done with LSDB. We will show how to filter the columns and how to filter the data.

First we will instantiate the LSDB object with the catalog we want to query. Then we will use a `conesearch` or `box_search` method to get the data we want. 

Let's start with DR4.

In [8]:
dr4_links = splusdata.get_hipscats("dr4/dual")[0]

dual_margin = lsdb.read_hipscat(dr4_links[1])
dual = lsdb.read_hipscat(
    dr4_links[0],
    margin_cache=dual_margin,
    columns=["ID", "RA", "DEC", 
             "r_auto", "g_auto", "i_auto", "z_auto", "u_auto",
             "J0378_auto", "J0395_auto", "J0410_auto", "J0430_auto",
             "J0515_auto", "J0660_auto", "J0861_auto"],
    filters=[("r_auto", "<", 18), ("g_auto", ">", 14)],
)

In [10]:
dual.cone_search(
    23, # ra 
    1, # dec
    1*3600 # radius in arcseconds
).compute()

,ID,RA,DEC,r_auto,g_auto,i_auto,z_auto,u_auto,J0378_auto,J0395_auto,J0410_auto,J0430_auto,J0515_auto,J0660_auto,J0861_auto
_hipscat_index,,,,,,,,,,,,,,,
5019946696994455552,DR4_3_STRIPE82-0036_0001237,23.32348,0.095314,17.327717,18.424343,16.738289,16.540657,99.0,18.166468,17.891554,16.945847,17.940763,17.531784,17.0086,15.607663
5019946701838876672,DR4_3_STRIPE82-0036_0001322,23.325101,0.098247,17.901476,19.286633,17.044563,16.800449,21.013481,18.13236,18.208101,17.652231,18.248005,18.2976,17.300993,15.909671
5019946708080001024,DR4_3_STRIPE82-0036_0001236,23.323378,0.099017,17.117977,18.252464,16.241793,15.727602,99.0,17.698477,17.193613,16.841116,17.134878,17.359024,16.432592,15.098984
5019946710097461248,DR4_3_STRIPE82-0036_0001326,23.323143,0.100581,17.169542,18.385048,16.150099,15.568332,20.059423,18.034189,17.510506,16.562744,17.474531,17.269455,16.471819,15.084396
5019946711703879680,DR4_3_STRIPE82-0036_0001405,23.323768,0.10228,16.834469,18.188778,15.874294,15.155374,21.012041,17.529688,17.14897,16.196367,17.092365,17.39735,16.113184,14.77909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5117251849790947328,DR4_3_STRIPE82-0036_0014055,23.798556,1.42368,15.018927,16.35486,13.913931,13.460277,19.127176,18.578876,18.478643,17.327932,17.383909,16.16489,14.640316,13.575714
5117252874790764544,DR4_3_STRIPE82-0036_0014359,23.696787,1.410547,17.243887,17.872732,17.049198,17.044365,19.217907,19.070717,18.820202,18.429239,18.274607,17.709839,17.204456,17.029226
5117253173123219456,DR4_3_STRIPE82-0036_0014227,23.656134,1.415943,15.969511,16.412558,15.846977,15.846822,17.533047,17.2722,17.267157,16.721003,16.691015,16.229187,15.964084,15.836373


In [17]:
dual.box_search(
    [-1, 1], # start RA, end RA
    [-1, 1] # start Dec, end Dec
).compute()

,ID,RA,DEC,r_auto,g_auto,i_auto,z_auto,u_auto,J0378_auto,J0395_auto,J0410_auto,J0430_auto,J0515_auto,J0660_auto,J0861_auto
_hipscat_index,,,,,,,,,,,,,,,
4899208249879822336,DR4_3_STRIPE82-0003_0011778,0.964254,-0.984355,16.980633,17.421703,16.841064,16.79199,18.530684,18.187626,18.122524,17.756912,17.710367,17.297626,16.964914,16.806829
4899211249155309568,DR4_3_STRIPE82-0003_0012430,0.97616,-0.96809,15.041121,15.501884,14.899343,14.86096,16.781298,16.417391,16.413391,15.887129,15.844776,15.341377,15.024549,14.879663
4899211261645946880,DR4_3_STRIPE82-0003_0012469,0.97898,-0.962708,16.135672,16.698135,15.943693,15.875214,18.214027,17.91004,17.767872,17.175526,17.187166,16.529642,16.089125,15.906317
4899278485723283456,DR4_3_STRIPE82-0001_0011810,0.276649,-0.991464,15.448023,15.887675,15.306561,15.257038,17.019674,16.641893,16.630896,16.210527,16.190672,15.736286,15.410069,15.257417
4899280014878441472,DR4_3_STRIPE82-0001_0012228,0.402118,-0.977255,17.930283,18.907173,17.592834,17.325251,20.071085,19.379517,19.606922,19.761612,19.422527,18.510283,17.860935,17.489532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477015167298437120,DR4_3_STRIPE82-0002_0044530,359.760099,0.998284,14.808131,15.016464,14.793859,14.838316,16.174797,15.619889,15.43687,15.16936,15.122585,14.894634,14.849438,14.831547
5477019443450085376,DR4_3_STRIPE82-0002_0040702,359.543212,0.988262,16.676786,17.273762,16.509445,16.448231,18.74291,18.595207,18.740599,17.786034,17.674099,17.059698,16.627821,16.473158
5477019576694734848,DR4_3_STRIPE82-0002_0045080,359.581452,0.997842,16.178837,16.673311,16.02849,15.98457,17.960754,17.596251,17.603821,17.060562,17.005465,16.486736,16.147192,15.973917


### Matching hipscat tables

Assuming you have the the table from the example before, lets instantiate a new hipscat object pointing to another table, in this case Star Galaxy Quasar (sqg) separation table, and make a match with the dual catalog.

First lets instantiate the hipscat that we want to match with the dual catalog. In this case we are using the sqg table.

In [18]:
dr4_sqg = splusdata.get_hipscats("dr4/sqg")[0]
sqg_margin = lsdb.read_hipscat(dr4_sqg[1])
sqg = lsdb.read_hipscat(
    dr4_sqg[0],
    margin_cache=sqg_margin,
    filters=[("PROB_GAL", ">", 0.8)]
)

/Users/gustavoschwarz/miniconda3/lib/python3.12/site-packages/hipscat/io/file_io/file_io.py:320: UserWarning: point_map.fits file written in RING ordering, due to https://github.com/astronomy-commons/hipscat/issues/271. Converting to NESTED.
  warnings.warn(


Then we can use the match method to match the sqg table with the dual catalog. The match method will return a new hipscat object with the matched sources.

In [ ]:
dual_sqg = sqg.crossmatch(
    dual, radius_arcsec = 1
)

In [20]:
result_table = dual_sqg.cone_search(
    1, #ra
    1, #dec
    1 * 3600 # radius in arcsecs
).compute()


In [21]:
result_table

,ID_sqg,RA_sqg,DEC_sqg,CLASS_sqg,model_flag_sqg,PROB_QSO_sqg,PROB_STAR_sqg,PROB_GAL_sqg,Norder_sqg,Dir_sqg,...,z_auto_dual,u_auto_dual,J0378_auto_dual,J0395_auto_dual,J0410_auto_dual,J0430_auto_dual,J0515_auto_dual,J0660_auto_dual,J0861_auto_dual,_dist_arcsec
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
5091982615334879232,DR4_3_STRIPE82-0003_0027945,1.150865,0.028838,2.0,0,0.0,0.0,1.0,5,0,...,16.79575,19.050779,18.839561,17.938129,18.594076,18.210499,17.890163,17.360659,16.743773,0.000282
5091982615334879233,DR4_3_STRIPE82-0004_0000309,1.150864,0.028841,2.0,0,0.0,0.01,0.99,5,0,...,16.846527,99.0,21.545265,99.0,20.170612,21.277454,18.713846,17.550707,16.940117,0.000282
5091984044355223552,DR4_3_STRIPE82-0003_0032189,1.204843,0.022827,2.0,0,0.0,0.0,1.0,5,0,...,16.89802,18.517649,18.489357,17.876053,17.87262,18.106815,17.605772,17.325048,16.743799,0.000282
5091985502148493312,DR4_3_STRIPE82-0004_0002515,1.293757,0.09462,2.0,0,0.0,0.0,1.0,5,0,...,16.339067,19.679325,19.227543,18.749567,19.539658,18.906784,17.509869,16.961966,16.473761,0.000282
5091996295271284736,DR4_3_STRIPE82-0003_0041728,0.935803,0.038474,2.0,1,0.0,0.03,0.97,5,0,...,17.094036,99.0,99.0,99.0,21.31982,99.0,19.08293,17.274836,17.269068,0.000282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477516506839384064,DR4_3_STRIPE82-0004_0031617,1.408031,1.376836,2.0,0,0.0,0.0,1.0,1,0,...,17.315907,19.482903,19.000202,18.59066,18.672577,18.517576,17.878046,17.745064,17.27664,0.000282
5477522346325573632,DR4_3_STRIPE82-0004_0030245,1.605574,1.412259,2.0,0,0.0,0.0,1.0,1,0,...,17.270304,20.656267,19.574299,19.082758,19.064711,18.794336,18.104109,17.721167,17.305927,0.000282
5477523454678794240,DR4_3_STRIPE82-0004_0030129,1.515884,1.415197,2.0,0,0.0,0.0,1.0,1,0,...,17.266359,19.315477,19.244976,99.0,20.037548,19.005175,18.162382,17.910954,17.321766,0.000282


Now imagine that we want to match our dual_sqg table with gaia dr3. We can do the same thing, instantiate a new hipscat object with the gaia dr3 table and use the match method to match the sources. 

Luckily some already partitioned catalogs are available at lsdb.io. We can use them to match with our dual catalog.

In [ ]:
gaia = lsdb.read_hipscat('https://data.lsdb.io/unstable/gaia_dr3/gaia/')

dual_gaia_sqg = dual_sqg.crossmatch(
    gaia, radius_arcsec=1
)

Now let's perform the `cone_search` method to get the data we want.

In [ ]:
dual_gaia_sqg.cone_search(
    1, #ra
    1, #dec
    1 * 3600 # radius in arcsecs
).compute()

### Matching an local table with hipscat

Of course you may also load your own table and match with some S-PLUS table.

We may open our table with pandas and then load it into a hipscat object. Then we can use the match method to match the sources.

In [ ]:
import pandas as pd

df = pd.DataFrame("table.csv")

# load it into lsdb
df_hips = lsdb.from_dataframe(df, ra_column="ra", dec_column="dec", margin_threshold=3600)

Then we can perform the crossmatch with the dual catalog for example.

In [ ]:
matched_table = df_hips.crossmatch(dual, radius_arcsec = 1).compute()